## Training Model from Scratch
Lisa Fung, 05/14/2024

Following Andrej Karpathy’s Recipe to Training Neural Networks: https://karpathy.github.io/2019/04/25/recipe/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.preprocessing import StandardScaler

from torchvision.io import read_image

In [2]:
BASE_PATH = "/kaggle/input/planttraits2024"
df = pd.read_csv(f'{BASE_PATH}/train.csv') # Ancillary geodata
df['image_path'] = f'{BASE_PATH}/train_images/'+df['id'].astype(str)+'.jpeg' # Image path

In [3]:
# Borrowed from Plant Traits Sample Notebook
class CFG:
    verbose = 1  # Verbosity
    seed = 42  # Random seed
    preset = "efficientnetv2_b2_imagenet"  # Name of pretrained classifier
    image_size = [224, 224]  # Input image size
    epochs = 12 # Training epochs
    batch_size = 96  # Batch size
    lr_mode = "step" # LR scheduler mode from one of "cos", "step", "exp"
    drop_remainder = True  # Drop incomplete batches
    num_classes = 6 # Number of classes in the dataset
    num_folds = 5 # Number of folds to split the dataset
    fold = 0 # Which fold to set as validation data
    class_names = ['X4_mean', 'X11_mean', 'X18_mean',
                   'X26_mean', 'X50_mean', 'X3112_mean',]
    aux_class_names = list(map(lambda x: x.replace("mean","sd"), class_names))
    num_classes = len(class_names)
    aux_num_classes = len(aux_class_names)

## Train/Validation Split
Borrowed from Plant Traits Sample Notebook

In [4]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=CFG.num_folds, shuffle=True, random_state=42)

# Create separate bin for each traits
for i, trait in enumerate(CFG.class_names):

    # Determine the bin edges dynamically based on the distribution of traits
    bin_edges = np.percentile(df[trait], np.linspace(0, 100, CFG.num_folds + 1))
    df[f"bin_{i}"] = np.digitize(df[trait], bin_edges)

# Concatenate the bins into a final bin
df["final_bin"] = (
    df[[f"bin_{i}" for i in range(len(CFG.class_names))]]
    .astype(str)
    .agg("".join, axis=1)
)

# Perform the stratified split using final bin
df = df.reset_index(drop=True)
for fold, (train_idx, valid_idx) in enumerate(skf.split(df, df["final_bin"])):
    df.loc[valid_idx, "fold"] = fold

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [5]:
train_df = df[df["fold"] != 0]
valid_df = df[df["fold"] == 0] # Fold 0 is validation
train_df[CFG.class_names + ["fold"]].describe()

,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean,fold
count,44391.000000,4.439100e+04,4.439100e+04,4.439100e+04,44391.000000,4.439100e+04,44391.000000
mean,0.522456,1.271709e+02,2.460040e+04,3.096704e+03,12.810444,4.938293e+05,2.499966
std,0.176001,1.237979e+04,2.582362e+06,2.210436e+05,1313.424294,1.023270e+08,1.118037
min,-2.431157,6.780000e-05,2.330000e-08,5.500000e-07,0.000097,7.690000e-08,1.000000
25%,0.410739,1.063560e+01,3.099867e-01,5.595144e-01,1.174045,2.552807e+02,1.500000
50%,0.509275,1.512003e+01,7.171231e-01,2.529542e+00,1.480130,7.258266e+02,2.000000
75%,0.622427,1.968705e+01,3.574691e+00,1.498396e+01,1.924787,2.158052e+03,3.000000
max,4.475172,1.504254e+06,2.720494e+08,3.106555e+07,159759.897700,2.155911e+10,4.000000


## Pre-Process Data
For each trait, we apply a combination of the following pre-processing techniques depending on the specific distribution of the trait's values. The pre-processing ONLY APPLIES to **training data** (NOT validation data), so we ignore fold 0.
1. Remove outliers by filtering for the quantile range (0.005, 0.995) (for all traits)
2. Use log base 10 transformation of right-skewed data (for all traits except trait X4)
3. Normalize data to have mean = 0 and standard deviation = 1 (for all traits)

In [6]:
class PlantDataPreProcess:
    lower_quantile = 0.005
    upper_quantile = 0.995
    log_transform = np.log10

In [7]:
# Filter data
print("Num samples before filering:", len(train_df))

for trait in CFG.class_names:
    lower_bound = train_df[trait].quantile(PlantDataPreProcess.lower_quantile)
    upper_bound = train_df[trait].quantile(PlantDataPreProcess.upper_quantile)
    train_df = train_df[(train_df[trait] >= lower_bound) & (train_df[trait] <= upper_bound)]
    
print("Num samples After filtering:", len(train_df))
train_df[CFG.class_names].describe()

Num samples before filering: 44391
Num samples After filtering: 41799


,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean
count,41799.000000,41799.000000,41799.000000,41799.000000,41799.000000,41799.000000
mean,0.521677,15.822802,3.207161,42.242341,1.616606,1858.228362
std,0.144269,7.598820,5.347785,166.643061,0.638061,3116.096582
min,0.176725,2.830246,0.032735,0.006451,0.494166,9.725925
25%,0.410762,10.792842,0.318074,0.585903,1.186274,267.237330
50%,0.509045,15.127512,0.714281,2.534105,1.481558,729.941079
75%,0.621267,19.509902,3.402668,14.288584,1.909773,2106.917500
max,0.957788,58.287012,32.388908,2369.101479,4.608223,29876.601410


In [8]:
# Log10 transformation for all traits except X4
y_train = train_df[CFG.class_names]

for skewed_trait in CFG.class_names[1:]:
    y_train.loc[:, skewed_trait] = y_train[skewed_trait].apply(PlantDataPreProcess.log_transform)
    
y_train.describe()

,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean
count,41799.000000,41799.000000,41799.000000,41799.000000,41799.000000,41799.000000
mean,0.521677,1.147506,0.002329,0.456349,0.177628,2.853320
std,0.144269,0.221707,0.667658,1.036389,0.163305,0.645702
min,0.176725,0.451824,-1.484983,-2.190374,-0.306127,0.987931
25%,0.410762,1.033136,-0.497472,-0.232174,0.074185,2.426897
50%,0.509045,1.179768,-0.146131,0.403825,0.170719,2.863288
75%,0.621267,1.290255,0.531820,1.154989,0.280982,3.323648
max,0.957788,1.765572,1.510396,3.374584,0.663534,4.475331


In [9]:
# Normalize to mean = 0, std dev = 1
from sklearn.preprocessing import StandardScaler

SCALER = StandardScaler()
y_train = SCALER.fit_transform(y_train)

y_train_df = pd.DataFrame(y_train, columns=CFG.class_names)
y_train_df.describe()

,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean
count,4.179900e+04,4.179900e+04,4.179900e+04,4.179900e+04,4.179900e+04,4.179900e+04
mean,-5.490689e-17,-3.829883e-16,-3.357310e-17,1.699904e-19,1.031842e-16,-4.310956e-16
std,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00,1.000012e+00
min,-2.391054e+00,-3.137878e+00,-2.227683e+00,-2.553825e+00,-2.962323e+00,-2.888967e+00
25%,-7.688166e-01,-5.158674e-01,-7.485979e-01,-6.643567e-01,-6.334448e-01,-6.604100e-01
50%,-8.755688e-02,1.455159e-01,-2.223625e-01,-5.068134e-02,-4.231292e-02,1.543735e-02
75%,6.903150e-01,6.438710e-01,7.930659e-01,6.741179e-01,6.328929e-01,7.284061e-01
max,3.022934e+00,2.787790e+00,2.258770e+00,2.815806e+00,2.975485e+00,2.512043e+00


## View Data

In [ ]:
df.columns.to_list()
df.describe()

In [ ]:
# Get images
def decode_image(inp, image_row):
    path = inp.loc[image_row, "image_path"]

    # Read jpeg image
    image = read_image(path)
    return image.permute(1, 2, 0)

#### Check Annual Mean Temperature below 0 degrees C

In [ ]:
# Get all images with very low annual mean temperature
below_0_temp_idx = df.index[df['WORLDCLIM_BIO1_annual_mean_temperature'] < 0]
len(below_0_temp_idx)

In [ ]:
# Check images
fig = plt.figure()

for i in range(20):
    fig.add_subplot(4, 5, i+1) 
    plt.imshow(decode_image(df, below_0_temp_idx[i]))

### Check Variation in Labels (Plant Traits)

In [ ]:
# Check variation of labels
# regex guide: https://www.w3schools.com/python/python_regex.asp
df.filter(regex="^X.+_mean$").describe()

#### Check X11 (Leaf area per leaf dry mass) outliers

In [ ]:
# X11: Leaf area per leaf dry mass
print("Total train samples:", len(df))
x11_greater_100_idx = df.index[df['X11_mean'] > 100]
x11_less_01_idx = df.index[df['X11_mean'] < 0.1]
print("X11 > 100:", len(x11_greater_100_idx))
print("X11 < 0.1:", len(x11_less_01_idx))

In [ ]:
# Check images
fig = plt.figure()

for i in range(20):
    fig.add_subplot(4, 5, i+1)
    plt.imshow(decode_image(df, x11_greater_100_idx[i]))
    
plt.title("X11 greater than 100")
# Big leaves, very little weight

In [ ]:
# Check images
fig = plt.figure()

for i in range(20):
    fig.add_subplot(4, 5, i+1)
    plt.imshow(decode_image(df, x11_less_01_idx[i]))
    
plt.title("X11 less than 0.1")
# Smaller, heavier leaves

### Visualize Distributions of Each Trait

In [ ]:
i = 2
col = CFG.class_names[i]
lower = 0.005
upper = 0.995
lower_q = df[col].quantile(lower)
upper_q = df[col].quantile(upper)

print(f"Original data for trait {i}")
print(df[col].describe())
print()
print(f"Lower quantile ({lower}):", lower_q)
print(f"Upper quantile ({upper}):", upper_q)
print("Unique values:", len(df[col].unique()))
print("Largest unique values:", sorted(df[col].unique(), reverse=True)[:5])
print()
print(f"Outliers removed data for trait {i}")
filtered_df = df[(df[col] >= lower_q) & (df[col] <= upper_q)][col]
transform_filtered_df = filtered_df.apply(np.log10)
print(transform_filtered_df.describe())
print(transform_filtered_df.hist())